# Recommendation System Pokemon

In [ ]:
import io
from google.colab import files
import pandas as pd
import numpy as np
import math

## The Data

In [ ]:
data_to_load = files.upload()

Saving Pokemon Database.csv to Pokemon Database.csv


Let's have a first look to our data.

In [ ]:
dataset=pd.read_csv(io.BytesIO(data_to_load['Pokemon Database.csv']))
dataset.head()

,Pokemon Id,Pokedex Number,Pokemon Name,Classification,Alternate Form Name,Original Pokemon ID,Legendary Type,Pokemon Height,Pokemon Weight,Primary Type,Secondary Type,Primary Ability,Primary Ability Description,Secondary Ability,Secondary Ability Description,Hidden Ability,Hidden Ability Description,Special Event Ability,Special Event Ability Description,Male Ratio,Female Ratio,Base Happiness,Game(s) of Origin,Region of Origin,Health Stat,Attack Stat,Defense Stat,Special Attack Stat,Special Defense Stat,Speed Stat,Base Stat Total,Health EV,Attack EV,Defense EV,Special Attack EV,Special Defense EV,Speed EV,EV Yield Total,Catch Rate,Experience Growth,Experience Growth Total,Experience Yield,Primary Egg Group,Secondary Egg Group,Egg Cycle Count,Pre-Evolution Pokemon Id,Evolution Details
0,1,1,Bulbasaur,Seed Pokemon,NaN,NaN,NaN,0.7,6.9,Grass,Poison,Overgrow,Powers up Grass-type moves when the Pokémon''s...,NaN,NaN,Chlorophyll,Boosts the Pokémon''s Speed stat in harsh sunl...,NaN,NaN,87.5,12.5,70,Red,Kanto,45,49,49,65,65,45,318,0,0,0,1,0,0,1,45,Medium Slow,1059860,64,Monster,Grass,20,NaN,NaN
1,2,2,Ivysaur,Seed Pokemon,NaN,NaN,NaN,1.0,13.0,Grass,Poison,Overgrow,Powers up Grass-type moves when the Pokémon''s...,NaN,NaN,Chlorophyll,Boosts the Pokémon''s Speed stat in harsh sunl...,NaN,NaN,87.5,12.5,70,Red,Kanto,60,62,63,80,80,60,405,0,0,0,1,1,0,2,45,Medium Slow,1059860,142,Monster,Grass,20,1.0,Level 16
2,3,3,Venusaur,Seed Pokemon,NaN,NaN,NaN,2.0,100.0,Grass,Poison,Overgrow,Powers up Grass-type moves when the Pokémon''s...,NaN,NaN,Chlorophyll,Boosts the Pokémon''s Speed stat in harsh sunl...,NaN,NaN,87.5,12.5,70,Red,Kanto,80,82,83,100,100,80,525,0,0,0,2,1,0,3,45,Medium Slow,1059860,236,Monster,Grass,20,2.0,Level 32
3,4,3,Venusaur,Seed Pokemon,Mega,3.0,NaN,2.4,155.5,Grass,Poison,Thick Fat,The Pokémon is protected by a layer of thick f...,NaN,NaN,NaN,NaN,NaN,NaN,87.5,12.5,70,X,Kalos,80,100,123,122,120,80,625,0,0,0,2,1,0,3,45,Medium Slow,1059860,281,Monster,Grass,20,NaN,NaN
4,1526,3,Venusaur,Seed Pokemon,Gigantamax,3.0,NaN,24.0,0.0,Grass,Poison,Overgrow,Powers up Grass-type moves when the Pokémon''s...,NaN,NaN,Chlorophyll,Boosts the Pokémon''s Speed stat in harsh sunl...,NaN,NaN,87.5,12.5,70,Sword,Galar,80,82,83,100,100,80,525,0,0,0,2,1,0,3,45,Medium Slow,1059860,236,Monster,Grass,20,NaN,NaN


We do not have an interest in the different forms of a Pokémon.

In [ ]:
dataset=dataset[dataset["Alternate Form Name"].isna()]

Let's have a look to the columns in this dataset.

In [ ]:
dataset.columns

Index(['Pokemon Id', 'Pokedex Number', 'Pokemon Name', 'Classification',
       'Alternate Form Name', 'Original Pokemon ID', 'Legendary Type',
       'Pokemon Height', 'Pokemon Weight', 'Primary Type', 'Secondary Type',
       'Primary Ability', 'Primary Ability Description', 'Secondary Ability',
       'Secondary Ability Description', 'Hidden Ability',
       'Hidden Ability Description', 'Special Event Ability',
       'Special Event Ability Description', 'Male Ratio', 'Female Ratio',
       'Base Happiness', 'Game(s) of Origin', 'Region of Origin',
       'Health Stat', 'Attack Stat', 'Defense Stat', 'Special Attack Stat',
       'Special Defense Stat', 'Speed Stat', 'Base Stat Total', 'Health EV',
       'Attack EV', 'Defense EV', 'Special Attack EV', 'Special Defense EV',
       'Speed EV', 'EV Yield Total', 'Catch Rate', 'Experience Growth',
       'Experience Growth Total', 'Experience Yield', 'Primary Egg Group',
       'Secondary Egg Group', 'Egg Cycle Count', 'Pre-Evolution 

We drop all unuseful columns.

In [ ]:
dataset.drop(columns=['Pokemon Id','Pokedex Number', 'Original Pokemon ID', 'Male Ratio',
                      'Female Ratio', 'Base Happiness', 'Health Stat', 'Attack Stat', 'Defense Stat', 
                      'Special Attack Stat', 'Special Defense Stat', 'Speed Stat', 'Base Stat Total', 
                      'Health EV','Attack EV', 'Defense EV', 'Special Attack EV', 'Special Defense EV',
                      'Speed EV', 'EV Yield Total', 'Catch Rate', 'Experience Growth',
                      'Experience Growth Total', 'Experience Yield', 'Primary Egg Group',
                      'Secondary Egg Group', 'Egg Cycle Count'],inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Let's have a new look at our data.

In [ ]:
dataset.head(5)

,Pokemon Name,Classification,Alternate Form Name,Legendary Type,Pokemon Height,Pokemon Weight,Primary Type,Secondary Type,Primary Ability,Primary Ability Description,Secondary Ability,Secondary Ability Description,Hidden Ability,Hidden Ability Description,Special Event Ability,Special Event Ability Description,Game(s) of Origin,Region of Origin,Pre-Evolution Pokemon Id,Evolution Details
0,Bulbasaur,Seed Pokemon,NaN,NaN,0.7,6.9,Grass,Poison,Overgrow,Powers up Grass-type moves when the Pokémon''s...,NaN,NaN,Chlorophyll,Boosts the Pokémon''s Speed stat in harsh sunl...,NaN,NaN,Red,Kanto,NaN,NaN
1,Ivysaur,Seed Pokemon,NaN,NaN,1.0,13.0,Grass,Poison,Overgrow,Powers up Grass-type moves when the Pokémon''s...,NaN,NaN,Chlorophyll,Boosts the Pokémon''s Speed stat in harsh sunl...,NaN,NaN,Red,Kanto,1.0,Level 16
2,Venusaur,Seed Pokemon,NaN,NaN,2.0,100.0,Grass,Poison,Overgrow,Powers up Grass-type moves when the Pokémon''s...,NaN,NaN,Chlorophyll,Boosts the Pokémon''s Speed stat in harsh sunl...,NaN,NaN,Red,Kanto,2.0,Level 32
5,Charmander,Lizard Pokemon,NaN,NaN,0.6,8.5,Fire,NaN,Blaze,Powers up Fire-type moves when the Pokémon''s ...,NaN,NaN,Solar Power,"Boosts the Sp. Atk stat in harsh sunlight, but...",NaN,NaN,Red,Kanto,NaN,NaN
6,Charmeleon,Flame Pokemon,NaN,NaN,1.1,19.0,Fire,NaN,Blaze,Powers up Fire-type moves when the Pokémon''s ...,NaN,NaN,Solar Power,"Boosts the Sp. Atk stat in harsh sunlight, but...",NaN,NaN,Red,Kanto,5.0,Level 16


In order to create our recommendation system, we need to select which columns are really iseful. 
- All the columns about its abilities and their descriptions are not suitable for the purpose of this exercise.
- The weight and the pre-evolution Id are not useful.

In [ ]:
dataset.drop(columns=[ 'Pokemon Weight','Alternate Form Name',
                      'Primary Ability', 'Primary Ability Description', 'Secondary Ability',
                      'Secondary Ability Description', 'Hidden Ability',
                      'Hidden Ability Description', 'Special Event Ability',
                      'Special Event Ability Description','Evolution Details'],
             inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


## First Step : Select the right columns

This is our final dataset, let's preprocess the data.

In [ ]:
dataset.head(5)

,Pokemon Name,Classification,Legendary Type,Pokemon Height,Primary Type,Secondary Type,Game(s) of Origin,Region of Origin,Pre-Evolution Pokemon Id
0,Bulbasaur,Seed Pokemon,NaN,0.7,Grass,Poison,Red,Kanto,NaN
1,Ivysaur,Seed Pokemon,NaN,1.0,Grass,Poison,Red,Kanto,1.0
2,Venusaur,Seed Pokemon,NaN,2.0,Grass,Poison,Red,Kanto,2.0
5,Charmander,Lizard Pokemon,NaN,0.6,Fire,NaN,Red,Kanto,NaN
6,Charmeleon,Flame Pokemon,NaN,1.1,Fire,NaN,Red,Kanto,5.0


### Column Legendary Type

In [ ]:
dataset["Legendary Type"].unique()

array([nan, 'Sub-Legendary', 'Legendary', 'Mythical'], dtype=object)

Our user must be able to choose if he prefers legendary pokemon or regular pokemon. We simplify this column by considering sub-legendary pokemon as legendary pokemon.

In [ ]:
dataset["Legendary Type"].replace({np.NaN:"Normal","Sub-Legendary":"Legendary"},inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [ ]:
dataset["Legendary Type"].unique()

array(['Normal', 'Legendary', 'Mythical'], dtype=object)

After the transformation, the column "Legendary Type" contains 3 categories:
- "Normal" which represents a common pokemon.
- "Mythical" which represents a rare pokemon.
- "Legendary" which represents a rare and very powerful pokemon.

In [ ]:
nb_categories_legendaryType=len(dataset["Legendary Type"].unique())

We stock in a variable the number of category for futher preprocessing.

### Column Pokemon Height

We want to give to our user the possibility to choose between a massive pokemon or a little one. We define a massive pokemon as a pokemon taller than a human.

In [ ]:
def height(value):
  if value>=1.85:
    return 1
  else :
    return 0

In [ ]:
dataset["Pokemon Height"]=dataset["Pokemon Height"].apply(height)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### Column Evolution Details

We return 0 if the Pokémon do not have an evolution or if the Pokémon has not evolve yet.

In [ ]:
def evolutionD(value):
  if value!=value:
    return 0
  else :
    return 1

In [ ]:
dataset["Pre-Evolution Pokemon Id"]=dataset["Pre-Evolution Pokemon Id"].apply(evolutionD)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### Column Secondary type

Some pokemons don't have a second type, we change this from "Nan" to "None".

In [ ]:
dataset["Secondary Type"].replace({np.NaN:"None"},inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


Let's check the differents types we can observed in the dataset.

In [ ]:
dataset["Secondary Type"].unique()

array(['Poison', 'None', 'Flying', 'Ground', 'Fairy', 'Grass', 'Fighting',
       'Psychic', 'Steel', 'Ice', 'Rock', 'Water', 'Electric', 'Fire',
       'Dragon', 'Dark', 'Ghost', 'Bug', 'Normal'], dtype=object)

We stock in a variable the number of type in this column and also the number of type in the column "Primary type" for futher usage.

In [ ]:
nb_categories_PrimaryType=len(dataset["Primary Type"].unique())

In [ ]:
nb_categories_SecondaryType=len(dataset["Secondary Type"].unique())

### One hot encoding columns

In [ ]:
dataset.columns

Index(['Pokemon Name', 'Classification', 'Legendary Type', 'Pokemon Height',
       'Primary Type', 'Secondary Type', 'Game(s) of Origin',
       'Region of Origin', 'Pre-Evolution Pokemon Id'],
      dtype='object')

In [ ]:
dataset=pd.get_dummies(dataset,columns=['Legendary Type','Game(s) of Origin', 'Region of Origin','Primary Type',
       'Secondary Type','Classification'])

Let's check how our dataset look now.

In [ ]:
dataset

,Pokemon Name,Pokemon Height,Pre-Evolution Pokemon Id,Legendary Type_Legendary,Legendary Type_Mythical,Legendary Type_Normal,Game(s) of Origin_Black,Game(s) of Origin_Diamond,Game(s) of Origin_Gold,Game(s) of Origin_Let''s Go Pikachu,Game(s) of Origin_Red,Game(s) of Origin_Ruby,Game(s) of Origin_Sun,Game(s) of Origin_Sword,Game(s) of Origin_Ultra Sun,Game(s) of Origin_X,Region of Origin_Alola,Region of Origin_Galar,Region of Origin_Hoenn,Region of Origin_Johto,Region of Origin_Kalos,Region of Origin_Kanto,Region of Origin_Sinnoh,Region of Origin_Unova,Primary Type_Bug,Primary Type_Dark,Primary Type_Dragon,Primary Type_Electric,Primary Type_Fairy,Primary Type_Fighting,Primary Type_Fire,Primary Type_Flying,Primary Type_Ghost,Primary Type_Grass,Primary Type_Ground,Primary Type_Ice,Primary Type_Normal,Primary Type_Poison,Primary Type_Psychic,Primary Type_Rock,...,Classification_Vibration Pokemon,Classification_Victory Pokemon,Classification_Viking Pokemon,Classification_Vine Pokemon,Classification_Virtual Pokemon,Classification_Volcano Pokemon,Classification_Warrior Pokemon,Classification_Water Bird Pokemon,Classification_Water Bubble Pokemon,Classification_Water Fish Pokemon,Classification_Water Gun Pokemon,Classification_Water Lizard Pokemon,Classification_Water Weed Pokemon,Classification_Weather Pokemon,Classification_Weed Pokemon,Classification_Whiskers Pokemon,Classification_Whisper Pokemon,Classification_White Bird Pokemon,Classification_Wicked Pokemon,Classification_Wild Bull Pokemon,Classification_Wild Duck Pokemon,Classification_Wild Horse Pokemon,Classification_Wild Monkey Pokemon,Classification_Wild Pigeon Pokemon,Classification_Willpower Pokemon,Classification_Wily Pokemon,Classification_Wind Chime Pokemon,Classification_Windveiled Pokemon,Classification_Wing Fish Pokemon,Classification_Wish Pokemon,Classification_Wolf Pokemon,Classification_Wood Gecko Pokemon,Classification_Woodpecker Pokemon,Classification_Wool Pokemon,Classification_Woolly Crab Pokemon,Classification_Worm Pokemon,Classification_Wrestling Pokemon,Classification_Wushu Pokemon,Classification_Young Fowl Pokemon,Classification_Zen Charm Pokemon
0,Bulbasaur,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Ivysaur,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Venusaur,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Charmander,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,Charmeleon,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1068,Zarude,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1069,Regieleki,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1070,Regidrago,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1071,Glastrier,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,

## Recommendation System - Test

In [ ]:
dataset

,Pokemon Name,Pokemon Height,Pre-Evolution Pokemon Id,Legendary Type_Legendary,Legendary Type_Mythical,Legendary Type_Normal,Game(s) of Origin_Black,Game(s) of Origin_Diamond,Game(s) of Origin_Gold,Game(s) of Origin_Let''s Go Pikachu,Game(s) of Origin_Red,Game(s) of Origin_Ruby,Game(s) of Origin_Sun,Game(s) of Origin_Sword,Game(s) of Origin_Ultra Sun,Game(s) of Origin_X,Region of Origin_Alola,Region of Origin_Galar,Region of Origin_Hoenn,Region of Origin_Johto,Region of Origin_Kalos,Region of Origin_Kanto,Region of Origin_Sinnoh,Region of Origin_Unova,Primary Type_Bug,Primary Type_Dark,Primary Type_Dragon,Primary Type_Electric,Primary Type_Fairy,Primary Type_Fighting,Primary Type_Fire,Primary Type_Flying,Primary Type_Ghost,Primary Type_Grass,Primary Type_Ground,Primary Type_Ice,Primary Type_Normal,Primary Type_Poison,Primary Type_Psychic,Primary Type_Rock,...,Classification_Vibration Pokemon,Classification_Victory Pokemon,Classification_Viking Pokemon,Classification_Vine Pokemon,Classification_Virtual Pokemon,Classification_Volcano Pokemon,Classification_Warrior Pokemon,Classification_Water Bird Pokemon,Classification_Water Bubble Pokemon,Classification_Water Fish Pokemon,Classification_Water Gun Pokemon,Classification_Water Lizard Pokemon,Classification_Water Weed Pokemon,Classification_Weather Pokemon,Classification_Weed Pokemon,Classification_Whiskers Pokemon,Classification_Whisper Pokemon,Classification_White Bird Pokemon,Classification_Wicked Pokemon,Classification_Wild Bull Pokemon,Classification_Wild Duck Pokemon,Classification_Wild Horse Pokemon,Classification_Wild Monkey Pokemon,Classification_Wild Pigeon Pokemon,Classification_Willpower Pokemon,Classification_Wily Pokemon,Classification_Wind Chime Pokemon,Classification_Windveiled Pokemon,Classification_Wing Fish Pokemon,Classification_Wish Pokemon,Classification_Wolf Pokemon,Classification_Wood Gecko Pokemon,Classification_Woodpecker Pokemon,Classification_Wool Pokemon,Classification_Woolly Crab Pokemon,Classification_Worm Pokemon,Classification_Wrestling Pokemon,Classification_Wushu Pokemon,Classification_Young Fowl Pokemon,Classification_Zen Charm Pokemon
0,Bulbasaur,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Ivysaur,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Venusaur,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Charmander,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,Charmeleon,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1068,Zarude,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1069,Regieleki,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1070,Regidrago,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1071,Glastrier,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,

We save the new version of the dataset in order to load it in our Chatbot.

In [ ]:
dataset.to_csv("/content/data_recommendation.csv")

In [ ]:
def cosine(u,m):
  u=np.array(u)
  m=np.array(m)
  numerator=u*m
  numerator=numerator.sum()
  u_norm=np.linalg.norm(u)
  m_norm=np.linalg.norm(m)
  denominator=u_norm*m_norm
  return numerator/denominator

In [ ]:
u=dataset[dataset["Pokemon Name"]=="Pikachu"]
u=u.iloc[:,1:]
results={}
v=dataset[dataset["Pokemon Name"]!="Pikachu"]
for index in range(len(v)):
    m=v.iloc[index,1:]
    name=v["Pokemon Name"].iloc[index]
    results[name]=cosine(u,m)
results_sort={k:v for k,v in sorted(results.items(), key=lambda item:item[1])}
for k in range(1,10):
    index=list(results_sort)[-k]
    print(index)


Raichu
Jolteon
Electabuzz
Electrode
Sandslash
Raticate
Voltorb
Sandshrew
Rattata
